![Zipline](https://media.quantopian.com/logos/open_source/zipline-logo-03_.png)

Zipline是QUANTOPIAN开发的算法交易库。这是一个事件驱动，支持回测和实时交易的系统。
Zipline目前有一个免费的[回测平台](https://www.quantopian.com)，可以很方便编写交易策略和执行回测。为处理A股数据，增加或修改基础数据相关部分，用于本机策略回测分析。

#### 功能

##### 新增`USEquityPricing`数据列

+ `prev_close` 前收盘
+ `turnover` 换手率
+ `amount` 成交额
+ `tmv` 总市值
+ `cmv` 流通市值
+ `circulating_share` 流通股本
+ `total_share` 总股本

**注意** 以上数据不会因股票分红派息而调整

##### 新增`Fundamentals`数据容器

`Fundamentals`是用于`pipeline`的数据集容器，包括偶发性变化及固定信息数据，如上市日期、行业分类、所属概念、财务数据等信息。

##### 提供后台自动提取、转换数据脚本

通过定时后台计划任务，自动提取回测所需网络数据，并转换为符合`zipline`需求的数据格式，可以更加专注于编写策略及相关分析。

#### 安装

##### 克隆

##### 安装包

##### 编译`C`扩展库

##### 安装`zipline`

#### 准备数据

成功安装`zipline`后，`cswd`包也已经成功安装。在相应环境下，执行以下命令：

>$ init-stock-data # 初始化基础数据。首次耗时大约4小时，以后每日后台自动刷新约半小时

>$ zipline ingest -b cndaily # 转换日线数据，耗时约10分钟

>$ sql-to-bcolz # `Fundamentals`数据，耗时约1.5分钟

#### 使用

计算`Fama-French`三因子案例涉及到：
1. 如何在`Notebook`运行回测
2. 选择基准收益率指数代码
3. 计划函数用法
4. `Fundamentals`及财务数据
5. `pipeline`及自定义因子用法
6. 回测速度

比较适合作为演示材料

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-1-1 --end 2018-1-1 --bm-symbol 399001
from zipline.api import symbol, sid, get_datetime

import pandas as pd
import numpy as np
from zipline.api import (attach_pipeline, pipeline_output, get_datetime,
                         calendars, schedule_function, date_rules)
from zipline.pipeline import Pipeline
from zipline.pipeline import CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.fundamentals import Fundamentals

# time frame on which we want to compute Fama-French
normal_days = 31
# approximate the number of trading days in that period
# this is the number of trading days we'll look back on,
# on every trading day.
business_days = int(0.69 * normal_days)


# 以下自定义因子选取期初数
class Returns(CustomFactor):
    """
    每个交易日每个股票窗口长度"business_days"期间收益率
    """
    window_length = business_days
    inputs = [USEquityPricing.close]

    def compute(self, today, assets, out, price):
        out[:] = (price[-1] - price[0]) / price[0] * 100


class MarketEquity(CustomFactor):
    """
    每个交易日每只股票所对应的总市值
    """
    window_length = business_days
    inputs = [USEquityPricing.tmv]

    def compute(self, today, assets, out, mcap):
        out[:] = mcap[0]


class BookEquity(CustomFactor):
    """
    每个交易日每只股票所对应的账面价值（所有者权益）
    """
    window_length = business_days
    inputs = [Fundamentals.balance_sheet.A107]

    def compute(self, today, assets, out, book):
        out[:] = book[0]


def initialize(context):
    """
    use our factors to add our pipes and screens.
    """
    pipe = Pipeline()
    mkt_cap = MarketEquity()
    pipe.add(mkt_cap, 'market_cap')

    book_equity = BookEquity()
    # book equity over market equity
    be_me = book_equity / mkt_cap
    pipe.add(be_me, 'be_me')

    returns = Returns()
    pipe.add(returns, 'returns')

    attach_pipeline(pipe, 'ff_example')
    schedule_function(
        func=myfunc,
        date_rule=date_rules.month_end())


def before_trading_start(context, data):
    """
    every trading day, we use our pipes to construct the Fama-French
    portfolios, and then calculate the Fama-French factors appropriately.
    """

    factors = pipeline_output('ff_example')

    # get the data we're going to use
    returns = factors['returns']
    mkt_cap = factors.sort_values(['market_cap'], ascending=True)
    be_me = factors.sort_values(['be_me'], ascending=True)

    # to compose the six portfolios, split our universe into portions
    half = int(len(mkt_cap) * 0.5)
    small_caps = mkt_cap[:half]
    big_caps = mkt_cap[half:]

    thirty = int(len(be_me) * 0.3)
    seventy = int(len(be_me) * 0.7)
    growth = be_me[:thirty]
    neutral = be_me[thirty:seventy]
    value = be_me[seventy:]

    # now use the portions to construct the portfolios.
    # note: these portfolios are just lists (indices) of equities
    small_value = small_caps.index.intersection(value.index)
    small_neutral = small_caps.index.intersection(neutral.index)
    small_growth = small_caps.index.intersection(growth.index)

    big_value = big_caps.index.intersection(value.index)
    big_neutral = big_caps.index.intersection(neutral.index)
    big_growth = big_caps.index.intersection(growth.index)

    # take the mean to get the portfolio return, assuming uniform
    # allocation to its constituent equities.
    sv = returns[small_value].mean()
    sn = returns[small_neutral].mean()
    sg = returns[small_growth].mean()

    bv = returns[big_value].mean()
    bn = returns[big_neutral].mean()
    bg = returns[big_growth].mean()

    # computing SMB
    context.smb = (sv + sn + sg) / 3 - (bv + bn + bg) / 3

    # computing HML
    context.hml = (sv + bv) / 2 - (sg + bg) / 2


def myfunc(context, data):
    d = get_datetime('Asia/Shanghai')
    print(d, context.smb, context.hml)

2017-01-26 15:00:00+08:00 0.014014289806335789 6.605843892342312
2017-02-28 15:00:00+08:00 4.1169182374497195 7.690119769984805
2017-03-31 15:00:00+08:00 0.35808304923773615 2.7492806758694215
2017-04-28 15:00:00+08:00 -4.318408584890385 5.414312699826368
2017-05-31 15:00:00+08:00 -0.4828317045367072 3.0869028143557147
2017-06-30 15:00:00+08:00 0.8640245866550513 0.09803178533289003
2017-07-31 15:00:00+08:00 -2.3024594948720227 6.2829537294457145
2017-08-31 15:00:00+08:00 3.2003154621799155 2.269609384481118
2017-09-29 15:00:00+08:00 1.1669055941862554 -0.6079568594636064
2017-10-31 15:00:00+08:00 -1.6233534895267374 -0.795885505339075
2017-11-30 15:00:00+08:00 -2.965097825507776 4.4434701009908615
2017-12-29 15:00:00+08:00 -1.1942883365086068 -0.38062423581176485
[2018-05-02 01:31:47.504203] INFO: zipline.finance.metrics.tracker: Simulated 244 trading days
first open: 2017-01-03 01:31:00+00:00
last close: 2017-12-29 07:00:00+00:00


,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.008422,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.020412,0.038928,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.019096,0.108456,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011028,0.143696,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.015197,0.124811,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.012696,0.117206,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.003768,0.125535,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,-0.004510,0.126307,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,-0.016590,0.133134,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,-0.045626,0.187327,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


**运行时长10-12秒**

+ 有关如何使用，请参考[quantopian使用手册](https://www.quantopian.com/help)。
+ 有关本项目的说明，请参阅[介绍材料](https://github.com/liudengfeng/czipline/tree/master/docs/%E4%BB%8B%E7%BB%8D%E6%9D%90%E6%96%99)
+ 有关后台自动数据处理，请参考[脚本](https://github.com/liudengfeng/czipline/blob/master/docs/%E4%BB%8B%E7%BB%8D%E6%9D%90%E6%96%99/bg_zipline_tasks.cron)

**特别说明**：个人当前使用Ubuntu17.10操作系统

#### 参考配置

![系统](./images/sys.png)

+ Ubuntu 17.10
+ Anaconda 
+ python 3.6

#### 交流

该项目纯属个人爱好，水平有限，欢迎加入来一起完善。

**添加个人微信，请务必备注`zipline`**

![联系方式](./images/ldf.png)